In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Movies")
sc = SparkContext(conf = conf) 

In [2]:
df_users = sc.textFile('./moviedata/users.csv')
df_zipcodes = sc.textFile('./moviedata/zipcodes.csv')
df_rating = sc.textFile('./moviedata/rating.csv')
df_movies = sc.textFile('./moviedata/movies.csv')

In [3]:
def clean_df_movies(x):
    y = x.split('\"')
    return y[0].split(',')[0:1] + y[2].split(',')[2:]

In [4]:
#New
#Pseudocode:
#for each genre:
#  get list of movies in the above genre
#  join list of movies with rating
#  joining this with users to get zipcodes
#  join this with zipcodes to get state
#  reduce on basis of states to get average rating
#  return the best average rating state(requirement of question)

In [5]:
df_users = sc.textFile('./moviedata/users.csv').map(lambda x: x.split(","))
df_zipcodes = sc.textFile('./moviedata/zipcodes.csv').map(lambda x: x.split(","))
df_rating = sc.textFile('./moviedata/rating.csv').map(lambda x: x.split(","))
df_movies = sc.textFile('./moviedata/movies.csv')

In [6]:
movieid_userid_rating = df_rating.map(lambda x: (x[1] , (x[0], float(x[2]))))

In [7]:
userid_zipcode = df_users.map(lambda x : (x[0], x[4]))

In [8]:
zipcode_state = df_zipcodes.map(lambda x: (x[0], x[3]))

In [9]:
movieid_userid_rating.take(5)

[(u'97', (u'253', 4.0)),
 (u'269', (u'284', 4.0)),
 (u'526', (u'106', 4.0)),
 (u'180', (u'121', 3.0)),
 (u'86', (u'62', 2.0))]

In [10]:
#Clean df_movies
df_movies_cleaned = df_movies.map(lambda x: clean_df_movies(x))

In [13]:
for genre_id in range(3, 20):
    #  get list of movies in the above genre
    movie_list = df_movies_cleaned.filter(lambda x : int(x[genre_id]) == 1).map(lambda x : (x[0], genre_id))

    #  join list of movies with rating
    userid_rating_list = movie_list.join(movieid_userid_rating).map(lambda x : (x[1][1][0], x[1][1][1]))

    #  joining this with users to get zipcodes
    zipcode_rating_list = userid_rating_list.join(userid_zipcode).map(lambda x : (x[1][1], x[1][0]))

    #  join this with zipcodes to get state
    state_rating_list = zipcode_rating_list.join(zipcode_state).map(lambda x : (x[1][1], (x[1][0], 1)))

    #  reduce on basis of states to get average rating
    state_avg_rating_tuple = state_rating_list.reduceByKey(lambda x, y : (x[0]+y[0], x[1]+y[1]))
    state_avg_rating = state_avg_rating_tuple.map(lambda x: (x[0], float(x[1][0])/x[1][1]))
    print state_avg_rating.takeOrdered(1, key = lambda x: -x[1])


[(u'"ME"', 4.5)]
[(u'"MS"', 5.0)]
[(u'"MS"', 4.444444444444445)]
[(u'"WV"', 4.666666666666667)]
[(u'"WV"', 4.833333333333333)]
[(u'"SD"', 5.0)]
[(u'"WV"', 4.767441860465116)]
[(u'"AL"', 5.0)]
[(u'"HI"', 5.0)]
[(u'"ME"', 5.0)]
[(u'"ME"', 4.375)]
[(u'"WV"', 4.666666666666667)]
[(u'"WV"', 4.5)]
[(u'"WV"', 4.476190476190476)]
[(u'"WV"', 4.5)]
[(u'"WV"', 4.684210526315789)]
[(u'"ME"', 4.333333333333333)]
